# Compilation

---

## Automate schema 
i made a dynamic way that allows me to create manopulate the automathe without change the code if the lexical analizer


In [9]:
import string

final_states = {
    9: "Keyword_programme",
    18: "Keyword_constante",
    26: "Keyword_caractere",
    34: "Keyword_variable",
    39: "Keyword_debut",
    42: "Keyword_fin",
    49: "Keyword_répéter",
    51: "Keyword_réel",
    58: "Keyword_jusqua",
    64: "Keyword_entier",
    
    65: "MultOperator_et",
    68: "MultOperator_mod",
    71: "MultOperator_div",
    69: "MultOperator_mult",
    
    73: "AddOperator_ou",
    78: "AddOperator_plus",
    79: "AddOperator_minus",
    80: "RelOperator_L",
    81: "RelOperator_LE",
    82: "RelOperator_NE",
    83: "RelOperator_G",
    84: "RelOperator_GE",
    85: "RelOperator_E",
    
    86: "Semicolon",
    
    75: "Name",
    
    77: "Number",
    
    88: "Float",
    
    89: "END_DOT",
    90: "COMMA",
    92: "ASSIGNMENT_OPERATOR",
    
    
}


# the states should be referred only by numbers 
rules = {
    0: {
        # Keywords
        "p": 1, "c": 10, "v": 27, "d": 35, "f": 40, "r": 43, "j": 52, ";": 86,
        ".": 89, ",": 90, ":": 91,
        
        # Relational Operators
        "<": 80, ">": 83, "=": 85,
        
        # Addition Operators
        "+": 78, 
        "-": 79,  
        "o": 72,
        
        # Multiplication Operators
        "e": 59, "m": 66, "*": 69, 
        
        # Name
        **{x: 74 for x in string.ascii_letters if x not in "pcvdfrjuoame"},
        
        # Numbers
        **{x: 77 for x in string.digits},
    },
    
    # Keywords
    1: {"r": 2,**{x: 75 for x in string.digits}},
    2: {"o": 3},
    3: {"g": 4},
    4: {"r": 5},
    5: {"a": 6},
    6: {"m": 7},
    7: {"m": 8},
    8: {"e": 9},  # programme_FOUND
    
    10: {"o": 11, "a": 19,**{x: 75 for x in string.digits}},
    11: {"n": 12},
    12: {"s": 13},
    13: {"t": 14},
    14: {"a": 15},
    15: {"n": 16},
    16: {"t": 17},
    17: {"e": 18},  # constante_FOUND
    
    19: {"r": 20},
    20: {"a": 21},
    21: {"c": 22},
    22: {"t": 23},
    23: {"è": 24},
    24: {"r": 25},
    25: {"e": 26},  # caractère_FOUND
    
    27: {"a": 28,**{x: 75 for x in string.digits}},
    28: {"r": 29},
    29: {"i": 30},
    30: {"a": 31},
    31: {"b": 32},
    32: {"l": 33},
    33: {"e": 34},  # variable_FOUND
    
    35: {"e": 36, "i": 70,**{x: 75 for x in string.digits}},
    36: {"b": 37},
    37: {"u": 38},
    38: {"t": 39},  # debut_FOUND
    70: {"v": 71},
    
    40: {"i": 41,**{x: 75 for x in string.digits}},
    41: {"n": 42},  # fin_FOUND
    
    43: {"é": 44, "e": 44,**{x: 75 for x in string.digits}},
    44: {"p": 45, "é": 50, "e": 50},
    45: {"é": 46, "e": 46},
    46: {"t": 47},
    47: {"e": 48},
    48: {"r": 48},  # répéter_FOUND
    
    50: {"l": 51},  # reel_FOUND
    
    52: {"u": 53,**{x: 75 for x in string.digits}},
    53: {"s": 54},
    54: {"q": 55},
    55: {"u": 56},
    56: {"'": 57},
    57: {"a": 58, "à": 58},  # jusqua_FOUND
    
    59: {"n": 60, "t": 65,**{x: 75 for x in string.digits}},
    60: {"t": 61},
    61: {"i": 62},
    62: {"e": 63},
    63: {"r": 64},  # entier_FOUND
    
    66: {"o": 67,**{x: 75 for x in string.digits}},
    67: {"d": 68},  # mod_FOUND
    
    72: {"u": 73,**{x: 75 for x in string.digits}},  # ou_FOUND
    
    87 :{**{x: 88 for x in string.digits}},
    
    91: {"=": 92},
    
    
    # Final states
    **{i: {} for i in final_states.keys()},
    
    # overide the final states with other states
    74: {**{x: 75 for x in string.digits}},
    75: {**{x: 76 for x in string.ascii_lowercase}},
    76: {**{x: 75 for x in string.digits}},
    77: {**{x: 77 for x in string.digits}, ".": 87},
    80: {">": 82, "=": 81},
    83: {"=": 84},
    88: {**{x: 88 for x in string.digits}},
}


---

## 2. Lexical Analizer

In [13]:

def lex(code):
    code = code.lower() 
    tokens = []
    state = 0
    buffer = ""
    position = 0

    while position < len(code):
        char = code[position]
        next_state = rules.get(state, {}).get(char)
        
        print(f"> Loop {position} =======================================================")
        print(f"char: {char}, state: {state}, next_state: {next_state}, buffer: `{buffer}`")

        if char.isspace():
            print(f"    > char is space")
            if state in final_states:
                tokens.append((final_states[state], buffer, position - len(buffer)))
                print(f"        > in final state : append token {final_states[state]} ; `{buffer}` ; {position - len(buffer)}")
            state = 0
            buffer = ""
        elif next_state is not None:
            state = next_state
            buffer += char
            
            print(f"    > char is not space and next state = {state} (not None)")
        else:
            print(f"    > char is not space and next state is None")
            if state in final_states:
                print(f"        > in final state : append token {final_states[state]} ; `{buffer}` ; {position - len(buffer)}")
                tokens.append((final_states[state], buffer, position - len(buffer)))
                state,buffer = 0,""
                if char != " ": position -=1
            elif state !=0:
                raise Exception(f"Lexical Error: Unexpected character '{char}' after `{buffer}` at position {position}")
            else :
                raise Exception(f"Lexical Error: Unexpected character '{char}' at position {position}")
            
            
        position += 1
        print(f"    > buffer: `{buffer}`")

    if state in final_states:
        tokens.append((final_states[state], buffer, position - len(buffer)))
    return tokens

code = "<2.2 22.34 >="
tokens = lex(code)
print(tokens)

> Loop 0 =======================================================
char: <, state: 0, next_state: 80, buffer: ``
    > char is not space and next state = 80 (not None)
    > buffer: `<`
> Loop 1 =======================================================
char: 2, state: 80, next_state: None, buffer: `<`
    > char is not space and next state is None
        > in final state : append token RelOperator_L ; `<` ; 0
    > buffer: ``
> Loop 1 =======================================================
char: 2, state: 0, next_state: 77, buffer: ``
    > char is not space and next state = 77 (not None)
    > buffer: `2`
> Loop 2 =======================================================
char: ., state: 77, next_state: 87, buffer: `2`
    > char is not space and next state = 87 (not None)
    > buffer: `2.`
> Loop 3 =======================================================
char: 2, state: 87, next_state: 88, buffer: `2.`
    > char is not space and next state = 88 (not None)
    > buffer: `2.2`
> Loop 4 ====

In [104]:

keywords = ["programme;", "<\nconstante12x3", "variable", "debut", "fin", "répéter", "jusqu'à", "div", "mod", "et", "ou", "entier", "caractère", "réel"]
text = " ".join(keywords)
print(text)
tokens = lex(text)
print(tokens)

programme; <
constante12x3 variable debut fin répéter jusqu'à div mod et ou entier caractère réel
> Loop 0 =======================================================
char: p, state: 0, next_state: 1, buffer: ``
    > char is not space and next state = 1 (not None)
    > buffer: `p`
> Loop 1 =======================================================
char: r, state: 1, next_state: 2, buffer: `p`
    > char is not space and next state = 2 (not None)
    > buffer: `pr`
> Loop 2 =======================================================
char: o, state: 2, next_state: 3, buffer: `pr`
    > char is not space and next state = 3 (not None)
    > buffer: `pro`
> Loop 3 =======================================================
char: g, state: 3, next_state: 4, buffer: `pro`
    > char is not space and next state = 4 (not None)
    > buffer: `prog`
> Loop 4 =======================================================
char: r, state: 4, next_state: 5, buffer: `prog`
    > char is not space and next state = 5 (not 

---

## 3. Syntaxic

In [14]:
class SyntaxAnalyzer:
    def __init__(self, tokens):
        self.tokens = tokens
        self.current_token = None
        self.index = 0
        self.next_token()
    
    def next_token(self):
        if self.index < len(self.tokens):
            self.current_token = self.tokens[self.index]
            self.index += 1
        else:
            self.current_token = None  # End of tokens

    def match(self, expected_type):
        if self.current_token and self.current_token.type == expected_type:
            self.next_token()
        else:
            raise SyntaxError(f"Expected {expected_type}, got {self.current_token}")

    def parse(self):
        self.parse_ProgrammeAlgoLang()

    def parse_ProgrammeAlgoLang(self):
        self.match("Keyword_programme")
        self.parse_NomProgramme()
        self.match("Semicolon")
        self.parse_Corps()
        self.match("Dot")

    def parse_Corps(self):
        if self.current_token.type == "Keyword_constante":
            self.parse_PartieDefinitionConstante()
        if self.current_token.type == "Keyword_variable":
            self.parse_PartieDefinitionVariable()
        self.parse_InstrComp()

    def parse_PartieDefinitionConstante(self):
        self.match("Keyword_constante")
        self.parse_DefinitionConstante()
        while self.current_token.type == "Name":
            self.parse_DefinitionConstante()

    def parse_DefinitionConstante(self):
        self.parse_NomConstante()
        self.match("Equals")
        self.parse_Constante()
        self.match("Semicolon")

    def parse_PartieDefinitionVariable(self):
        self.match("Keyword_variable")
        self.parse_DefinitionVariable()
        while self.current_token.type == "Name":
            self.parse_DefinitionVariable()

    def parse_DefinitionVariable(self):
        self.parse_GroupeVariable()
        self.match("Semicolon")

    def parse_GroupeVariable(self):
        self.parse_NomVariable()
        while self.current_token.type == "Comma":
            self.match("Comma")
            self.parse_NomVariable()
        self.match("Colon")
        self.parse_NomType()

    def parse_NomProgramme(self):
        self.parse_Nom()

    def parse_Nom(self):
        if self.current_token.type == "Name":
            self.match("Name")
        else:
            raise SyntaxError("Invalid name")

    def parse_Constante(self):
        if self.current_token.type == "Number":
            self.match("Number")
        elif self.current_token.type == "Name":
            self.match("Name")
        else:
            raise SyntaxError("Invalid constant")

    def parse_NomVariable(self):
        self.parse_Nom()

    def parse_NomType(self):
        if self.current_token.type in {"Keyword_entier", "Keyword_caractere", "Keyword_reel"}:
            self.match(self.current_token.type)
        else:
            raise SyntaxError("Invalid type")

    def parse_InstrComp(self):
        self.match("Keyword_debut")
        self.parse_Instruction()
        while self.current_token.type == "Semicolon":
            self.match("Semicolon")
            self.parse_Instruction()
        self.match("Keyword_fin")

    def parse_Instruction(self):
        if self.current_token.type == "Name":
            self.parse_InstructionAffectation()
        elif self.current_token.type == "Keyword_repeater":
            self.parse_InstructionRepeater()
        elif self.current_token.type == "Keyword_debut":
            self.parse_InstrComp()
        else:
            self.parse_Vide()

    def parse_InstructionAffectation(self):
        self.parse_NomVariable()
        self.match("Assign")
        self.parse_Expression()

    def parse_InstructionRepeater(self):
        self.match("Keyword_repeater")
        self.parse_Instruction()
        self.match("Keyword_jusqua")
        self.parse_Condition()

    def parse_Expression(self):
        self.parse_ExpressionSimple()
        if self.current_token.type in {"RelOperator_L", "RelOperator_LE", "RelOperator_G", "RelOperator_GE", "RelOperator_NE", "RelOperator_E"}:
            self.match(self.current_token.type)
            self.parse_ExpressionSimple()

    def parse_ExpressionSimple(self):
        if self.current_token.type in {"AddOperator_plus", "AddOperator_minus"}:
            self.match(self.current_token.type)
        self.parse_Terme()
        while self.current_token.type in {"AddOperator_plus", "AddOperator_minus", "AddOperator_ou"}:
            self.match(self.current_token.type)
            self.parse_Terme()

    def parse_Terme(self):
        self.parse_Facteur()
        while self.current_token.type in {"MultOperator_mult", "MultOperator_div", "MultOperator_mod", "MultOperator_et"}:
            self.match(self.current_token.type)
            self.parse_Facteur()

    def parse_Facteur(self):
        if self.current_token.type == "Number":
            self.match("Number")
        elif self.current_token.type == "Name":
            self.match("Name")
        elif self.current_token.type == "LeftParen":
            self.match("LeftParen")
            self.parse_Expression()
            self.match("RightParen")
        else:
            raise SyntaxError("Invalid factor")

    def parse_Condition(self):
        self.parse_Expression()
        self.parse_OperateurRelationnel()
        self.parse_Expression()

    def parse_OperateurRelationnel(self):
        if self.current_token.type in {"RelOperator_L", "RelOperator_LE", "RelOperator_G", "RelOperator_GE", "RelOperator_NE", "RelOperator_E"}:
            self.match(self.current_token.type)
        else:
            raise SyntaxError("Invalid relational operator")

    def parse_Vide(self):
        pass  # Empty production, do nothing


In [15]:
code = "programme v1a2 ;"
try:
    tokens = lex(code)
    for token in tokens:
        print(token)
    
    SyntaxAnalyzer(tokens).parse()
except ValueError as e:
    print(e)

> Loop 0 =======================================================
char: p, state: 0, next_state: 1, buffer: ``
    > char is not space and next state = 1 (not None)
    > buffer: `p`
> Loop 1 =======================================================
char: r, state: 1, next_state: 2, buffer: `p`
    > char is not space and next state = 2 (not None)
    > buffer: `pr`
> Loop 2 =======================================================
char: o, state: 2, next_state: 3, buffer: `pr`
    > char is not space and next state = 3 (not None)
    > buffer: `pro`
> Loop 3 =======================================================
char: g, state: 3, next_state: 4, buffer: `pro`
    > char is not space and next state = 4 (not None)
    > buffer: `prog`
> Loop 4 =======================================================
char: r, state: 4, next_state: 5, buffer: `prog`
    > char is not space and next state = 5 (not None)
    > buffer: `progr`
> Loop 5 =======================================================
char:

AttributeError: 'list' object has no attribute 'type'